# Introduction and Motivation

In the era of digital media, music streaming platforms have transformed how we discover and enjoy music. Services like Spotify have millions of tracks available at our fingertips, making music recommendation systems essential for enhancing user experience. Recognizing the significance of these systems, our team chose to explore the “Spotify Tracks, Genre, Audio Features” dataset from Kaggle (https://www.kaggle.com/datasets/pepepython/spotify-huge-database-daily-charts-over-3-years/data) using PySpark to analyze trends and build a music recommendation model.

Our motivation for selecting this dataset is rooted in a desire to engage with a more intriguing and relatable subject matter than the typical technical datasets we’ve encountered in previous classes. Music is a universal language, and understanding how recommendation systems work within this context is both exciting and relevant. Music recommendation models are increasingly important as the amount of data generated online continues to grow, impacting how users interact with streaming services.

The primary objectives of our project are twofold:
1. Data Analysis: We aim to understand trends within the music data by examining how different genres relate to specific audio feature metrics. For example, we want to investigate whether certain genres consistently exhibit higher danceability scores or how energy levels vary across genres.
2. Model Building: Utilizing machine learning techniques, we plan to develop a music recommendation model. This model will leverage the insights gained from our data analysis to suggest tracks that align with user preferences based on audio features.

Our project focuses on mainstream music, as the dataset predominantly features popular tracks. While we have considered algorithms like K-Nearest Neighbors for recommendation, we will determine the most suitable machine learning methods as we progress, ensuring they align with our dataset characteristics and project goals.

# Data Collection

In [51]:
from pyspark.sql import SparkSession

# Initialize a Spark session
# Start a new Spark session
spark = SparkSession.builder \
    .appName("MusicDataAnalysisProject") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

# Path to your CSV file
csv_path = "../Music_Database.csv"

music_df = spark.read.option("header", "true").csv(csv_path, inferSchema=True)
music_df = music_df.withColumnRenamed("Country", "Country0")

# Show the first few rows to verify
music_df.show()


+-----------+--------------------+------------------+---------------+--------------------+------------+-----------------+----------------+--------+--------------------+------------+------------+---------------+------------------+------------------+---+--------+----+-----------+-------------------+----------------+----------+-------+-------+-----------+--------------+---------+------------------+-------------------+--------------+-------------+-------+-----------+------+------+--------+---------+----------------+----+----+-------+-----+-----+----+-----+-----+-----+---+---+--------+---+------+---------+----+----+------------+---------+----------+--------+-------+----+-----+----+-----+------------+-------+----+----+-------+--------+-----+--------+--------+-------+----------+-------------------+------------+-------------------+-------------------+------------------+-------------+-------------------+------------------+-------------------+-----------+------------------+-------------+--------

24/11/14 15:33:33 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Country, Uri, Popularity, Title, Artist, Album/Single, Genre, Artist_followers, Explicit, Album, Release_date, Track_number, Tracks_in_album, danceability, energy, key, loudness, mode, speechiness, acoustics, instrumentalness, liveliness, valence, tempo, duration_ms, time_signature, Genre_new, Days_since_release, Released_after_2017, Explicit_false, Explicit_true, album, compilation, single, bolero, boy band, country, dance/electronic, else, funk, hip hop, house, indie, jazz, k-pop, latin, metal, opm, pop, r&b/soul, rap, reggae, reggaeton, rock, trap, syuzhet_norm, bing_norm, afinn_norm, nrc_norm, syuzhet, bing, afinn, nrc, anger, anticipation, disgust, fear, joy, sadness, surprise, trust, negative, positive, n_words, anger_norm, anticipation_norm, disgust_norm, fear_norm, joy_norm, sadness_norm, surprise_norm, trust_norm, negative_norm, positive_norm, anger_norm2, anticipation_norm2, disgust_n

# Data Inspection and Validation

Numerical columns are String in the schema, so we'll convert them to numeric types later on (Data Transformation).

In [52]:
# Print the schema of the DataFrame
music_df.printSchema()

root
 |-- Country0: string (nullable = true)
 |-- Uri: string (nullable = true)
 |-- Popularity: double (nullable = true)
 |-- Title: string (nullable = true)
 |-- Artist: string (nullable = true)
 |-- Album/Single: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Artist_followers: string (nullable = true)
 |-- Explicit: string (nullable = true)
 |-- Album9: string (nullable = true)
 |-- Release_date: string (nullable = true)
 |-- Track_number: string (nullable = true)
 |-- Tracks_in_album: string (nullable = true)
 |-- danceability: string (nullable = true)
 |-- energy: string (nullable = true)
 |-- key: string (nullable = true)
 |-- loudness: string (nullable = true)
 |-- mode: string (nullable = true)
 |-- speechiness: string (nullable = true)
 |-- acoustics: string (nullable = true)
 |-- instrumentalness: string (nullable = true)
 |-- liveliness: string (nullable = true)
 |-- valence: string (nullable = true)
 |-- tempo: string (nullable = true)
 |-- duration_ms: 

Check for null and duplicates in the dataset. Also, check the distribution of popularity scores because later we will categoprize the data by low, medium, and high popularity levels.

In [53]:
from pyspark.sql.functions import col, sum, count, when
# Count null values in each column
null_counts = music_df.select([count(when(col(c).isNull(), c)).alias(c) for c in music_df.columns])
null_counts.show()

# Check for duplicates
duplicate_count = music_df.count() - music_df.dropDuplicates().count()
print(f"Number of duplicate rows: {duplicate_count}")

# Check popularity range for validity (e.g., ensure no extreme outliers)
music_df.select("Popularity").describe().show()


24/11/14 15:33:34 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Country, Uri, Popularity, Title, Artist, Album/Single, Genre, Artist_followers, Explicit, Album, Release_date, Track_number, Tracks_in_album, danceability, energy, key, loudness, mode, speechiness, acoustics, instrumentalness, liveliness, valence, tempo, duration_ms, time_signature, Genre_new, Days_since_release, Released_after_2017, Explicit_false, Explicit_true, album, compilation, single, bolero, boy band, country, dance/electronic, else, funk, hip hop, house, indie, jazz, k-pop, latin, metal, opm, pop, r&b/soul, rap, reggae, reggaeton, rock, trap, syuzhet_norm, bing_norm, afinn_norm, nrc_norm, syuzhet, bing, afinn, nrc, anger, anticipation, disgust, fear, joy, sadness, surprise, trust, negative, positive, n_words, anger_norm, anticipation_norm, disgust_norm, fear_norm, joy_norm, sadness_norm, surprise_norm, trust_norm, negative_norm, positive_norm, anger_norm2, anticipation_norm2, disgust_n

+--------+---+----------+-----+------+------------+-----+----------------+--------+------+------------+------------+---------------+------------+------+---+--------+----+-----------+---------+----------------+----------+-------+-----+-----------+--------------+---------+------------------+-------------------+--------------+-------------+-------+-----------+------+------+--------+---------+----------------+----+----+-------+-----+-----+----+-----+-----+-----+---+---+--------+---+------+---------+----+----+------------+---------+----------+--------+-------+-----+-----+-----+-----+------------+-------+-----+-----+-------+--------+-----+--------+--------+-------+----------+-----------------+------------+---------+--------+------------+-------------+----------+-------------+-------------+-----------+------------------+-------------+----------+---------+-------------+--------------+-----------+--------------+--------------+---------------+---------------+-----+--------------+-------------+--

24/11/14 15:33:37 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Country, Uri, Popularity, Title, Artist, Album/Single, Genre, Artist_followers, Explicit, Album, Release_date, Track_number, Tracks_in_album, danceability, energy, key, loudness, mode, speechiness, acoustics, instrumentalness, liveliness, valence, tempo, duration_ms, time_signature, Genre_new, Days_since_release, Released_after_2017, Explicit_false, Explicit_true, album, compilation, single, bolero, boy band, country, dance/electronic, else, funk, hip hop, house, indie, jazz, k-pop, latin, metal, opm, pop, r&b/soul, rap, reggae, reggaeton, rock, trap, syuzhet_norm, bing_norm, afinn_norm, nrc_norm, syuzhet, bing, afinn, nrc, anger, anticipation, disgust, fear, joy, sadness, surprise, trust, negative, positive, n_words, anger_norm, anticipation_norm, disgust_norm, fear_norm, joy_norm, sadness_norm, surprise_norm, trust_norm, negative_norm, positive_norm, anger_norm2, anticipation_norm2, disgust_n

Number of duplicate rows: 0
+-------+------------------+
|summary|        Popularity|
+-------+------------------+
|  count|            170633|
|   mean| 5417.616264145855|
| stddev|13115.854525897726|
|    min|               0.8|
|    max|233766.89999999988|
+-------+------------------+



# Data Filtering

In [54]:
# Further refine by North America
filtered_df = music_df.filter((music_df.Country0 == "USA") | (music_df.Country0 == "Canada") | (music_df.Country0 == "Mexico"))

print(f"Number of rows: {filtered_df.count()}")


24/11/14 15:33:43 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Country
 Schema: Country0
Expected: Country0 but found: Country
CSV file: file:///Users/nate/Desktop/FALL24%202/ENSF612/Project/Music_Analytics_and_Recommendation/Music_Database.csv


Number of rows: 15113


# Data Transformation

In [55]:
from pyspark.sql.functions import col

# List of columns to cast to integer or double as appropriate
int_columns = ["Track_number", "Tracks_in_album", "time_signature", "Argentina", "Australia", "Austria", 
               "Belgium", "Brazil", "Canada", "Chile", "Colombia", "Costa Rica", "Denmark", "Ecuador", 
               "Finland", "France", "Germany", "Global", "Indonesia", "Ireland", "Italy", "Malaysia", 
               "Mexico", "Netherlands", "New Zealand", "Norway", "Peru", "Philippines", "Poland", 
               "Portugal", "Singapore", "Spain", "Sweden", "Switzerland", "Taiwan", "Turkey", "UK", "USA"]

double_columns = ["Popularity", "Artist_followers", "danceability", "energy", "key", "loudness", 
                  "mode", "speechiness", "acoustics", "instrumentalness", "liveliness", "valence", 
                  "tempo", "duration_ms", "Days_since_release", "syuzhet_norm", "bing_norm", "afinn_norm", 
                  "nrc_norm", "syuzhet", "bing", "afinn", "nrc", "anger", "anticipation", "disgust", "fear", 
                  "joy", "sadness", "surprise", "trust", "negative", "positive", "n_words", "anger_norm", 
                  "anticipation_norm", "disgust_norm", "fear_norm", "joy_norm", "sadness_norm", "surprise_norm", 
                  "trust_norm", "negative_norm", "positive_norm", "anger_norm2", "anticipation_norm2", 
                  "disgust_norm2", "fear_norm2", "joy_norm2", "sadness_norm2", "surprise_norm2", "trust_norm2", 
                  "negative_norm2", "positive_norm2", "negative_bog_jr", "positive_bog_jr", "Bayes", 
                  "Negative_Bayes", "Neutral_Bayes", "Positive_Bayes", "Desire", "Explore", "Fun", 
                  "Hope", "Love", "Nostalgia", "Thug", "bing_norm_negative"]

# Cast columns to integer
for column in int_columns:
    transformed_df = filtered_df.withColumn(column, col(column).cast("int"))

# Cast columns to double
for column in double_columns:
    transformed_df = filtered_df.withColumn(column, col(column).cast("double"))

# Verify the updated schema
transformed_df.printSchema()


root
 |-- Country0: string (nullable = true)
 |-- Uri: string (nullable = true)
 |-- Popularity: double (nullable = true)
 |-- Title: string (nullable = true)
 |-- Artist: string (nullable = true)
 |-- Album/Single: string (nullable = true)
 |-- Genre: string (nullable = true)
 |-- Artist_followers: string (nullable = true)
 |-- Explicit: string (nullable = true)
 |-- Album9: string (nullable = true)
 |-- Release_date: string (nullable = true)
 |-- Track_number: string (nullable = true)
 |-- Tracks_in_album: string (nullable = true)
 |-- danceability: string (nullable = true)
 |-- energy: string (nullable = true)
 |-- key: string (nullable = true)
 |-- loudness: string (nullable = true)
 |-- mode: string (nullable = true)
 |-- speechiness: string (nullable = true)
 |-- acoustics: string (nullable = true)
 |-- instrumentalness: string (nullable = true)
 |-- liveliness: string (nullable = true)
 |-- valence: string (nullable = true)
 |-- tempo: string (nullable = true)
 |-- duration_ms: 

In [56]:
# Define popularity categories
transformed_df = transformed_df.withColumn(
    "PopularityCategory",
    when(transformed_df.Popularity < 5000, "Low")
    .when((transformed_df.Popularity >= 5000) & (transformed_df.Popularity < 18000), "Medium")
    .otherwise("High")
)

# Model 1 (Popularity Predictor)

In [77]:
# Convert popularity category we made prior to an indexed label
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol="PopularityCategory", outputCol="label")
model1_df = indexer.fit(transformed_df).transform(transformed_df)

24/11/14 15:43:55 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Country, Popularity
 Schema: Country0, Popularity
Expected: Country0 but found: Country
CSV file: file:///Users/nate/Desktop/FALL24%202/ENSF612/Project/Music_Analytics_and_Recommendation/Music_Database.csv


In [78]:
# Selected features including danceability, energy, etc.
feature_columns = ["Popularity", "Artist_followers", "danceability", "energy", "key", "loudness", 
                  "mode", "speechiness", "acoustics", "instrumentalness", "liveliness", "valence", 
                  "tempo", "duration_ms", "Days_since_release", "syuzhet_norm", "bing_norm", "afinn_norm", 
                  "nrc_norm", "syuzhet", "bing", "afinn", "nrc", "anger", "anticipation", "disgust", "fear", 
                  "joy", "sadness", "surprise", "trust", "negative", "positive", "n_words", "anger_norm", 
                  "anticipation_norm", "disgust_norm", "fear_norm", "joy_norm", "sadness_norm", "surprise_norm", 
                  "trust_norm", "negative_norm", "positive_norm", "anger_norm2", "anticipation_norm2", 
                  "disgust_norm2", "fear_norm2", "joy_norm2", "sadness_norm2", "surprise_norm2", "trust_norm2", 
                  "negative_norm2", "positive_norm2", "negative_bog_jr", "positive_bog_jr", "Bayes", 
                  "Negative_Bayes", "Neutral_Bayes", "Positive_Bayes", "Desire", "Explore", "Fun", 
                  "Hope", "Love", "Nostalgia", "Thug", "bing_norm_negative"]

# Ensure each feature is cast to double
for feature in feature_columns:
    model1_df = model1_df.withColumn(feature, col(feature).cast("double"))

# Fill null values in each feature column with 0
model1_df = model1_df.dropna()

# Assemble features
from pyspark.ml.feature import VectorAssembler

assembler = VectorAssembler(inputCols=feature_columns, outputCol="features", handleInvalid = "keep")
model1_df = assembler.transform(model1_df).select("features", "label")
model1_df.select("features").show(truncate=False)


+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|features                                                                                                                                                                                                                                                                                                                                               

24/11/14 15:44:07 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Country, Uri, Popularity, Title, Artist, Album/Single, Genre, Artist_followers, Explicit, Album, Release_date, Track_number, Tracks_in_album, danceability, energy, key, loudness, mode, speechiness, acoustics, instrumentalness, liveliness, valence, tempo, duration_ms, time_signature, Genre_new, Days_since_release, Released_after_2017, Explicit_false, Explicit_true, album, compilation, single, bolero, boy band, country, dance/electronic, else, funk, hip hop, house, indie, jazz, k-pop, latin, metal, opm, pop, r&b/soul, rap, reggae, reggaeton, rock, trap, syuzhet_norm, bing_norm, afinn_norm, nrc_norm, syuzhet, bing, afinn, nrc, anger, anticipation, disgust, fear, joy, sadness, surprise, trust, negative, positive, n_words, anger_norm, anticipation_norm, disgust_norm, fear_norm, joy_norm, sadness_norm, surprise_norm, trust_norm, negative_norm, positive_norm, anger_norm2, anticipation_norm2, disgust_n

In [79]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Split data into train and test sets
train_df, test_df = model1_df.randomSplit([0.8, 0.2], seed=42)

# Initialize and train the Random Forest classifier
rf_classifier = RandomForestClassifier(featuresCol="features", labelCol="label", numTrees=100, maxDepth=10)
rf_model = rf_classifier.fit(train_df)

# Make predictions
predictions = rf_model.transform(test_df)


24/11/14 15:44:07 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Country, Uri, Popularity, Title, Artist, Album/Single, Genre, Artist_followers, Explicit, Album, Release_date, Track_number, Tracks_in_album, danceability, energy, key, loudness, mode, speechiness, acoustics, instrumentalness, liveliness, valence, tempo, duration_ms, time_signature, Genre_new, Days_since_release, Released_after_2017, Explicit_false, Explicit_true, album, compilation, single, bolero, boy band, country, dance/electronic, else, funk, hip hop, house, indie, jazz, k-pop, latin, metal, opm, pop, r&b/soul, rap, reggae, reggaeton, rock, trap, syuzhet_norm, bing_norm, afinn_norm, nrc_norm, syuzhet, bing, afinn, nrc, anger, anticipation, disgust, fear, joy, sadness, surprise, trust, negative, positive, n_words, anger_norm, anticipation_norm, disgust_norm, fear_norm, joy_norm, sadness_norm, surprise_norm, trust_norm, negative_norm, positive_norm, anger_norm2, anticipation_norm2, disgust_n

In [80]:
# Evaluate the model
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print(f"Accuracy of the classifier: {accuracy}")

# Display a sample of predictions
predictions.select("prediction", "label", "features").show(5)


24/11/14 15:44:26 WARN DAGScheduler: Broadcasting large task binary with size 2.9 MiB
24/11/14 15:44:27 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Country, Uri, Popularity, Title, Artist, Album/Single, Genre, Artist_followers, Explicit, Album, Release_date, Track_number, Tracks_in_album, danceability, energy, key, loudness, mode, speechiness, acoustics, instrumentalness, liveliness, valence, tempo, duration_ms, time_signature, Genre_new, Days_since_release, Released_after_2017, Explicit_false, Explicit_true, album, compilation, single, bolero, boy band, country, dance/electronic, else, funk, hip hop, house, indie, jazz, k-pop, latin, metal, opm, pop, r&b/soul, rap, reggae, reggaeton, rock, trap, syuzhet_norm, bing_norm, afinn_norm, nrc_norm, syuzhet, bing, afinn, nrc, anger, anticipation, disgust, fear, joy, sadness, surprise, trust, negative, positive, n_words, anger_norm, anticipation_norm, disgust_norm, fear_norm, joy_norm, sadness_norm, surprise_norm

Accuracy of the classifier: 0.9675767918088737


24/11/14 15:44:29 WARN DAGScheduler: Broadcasting large task binary with size 2.9 MiB
24/11/14 15:44:29 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Country, Uri, Popularity, Title, Artist, Album/Single, Genre, Artist_followers, Explicit, Album, Release_date, Track_number, Tracks_in_album, danceability, energy, key, loudness, mode, speechiness, acoustics, instrumentalness, liveliness, valence, tempo, duration_ms, time_signature, Genre_new, Days_since_release, Released_after_2017, Explicit_false, Explicit_true, album, compilation, single, bolero, boy band, country, dance/electronic, else, funk, hip hop, house, indie, jazz, k-pop, latin, metal, opm, pop, r&b/soul, rap, reggae, reggaeton, rock, trap, syuzhet_norm, bing_norm, afinn_norm, nrc_norm, syuzhet, bing, afinn, nrc, anger, anticipation, disgust, fear, joy, sadness, surprise, trust, negative, positive, n_words, anger_norm, anticipation_norm, disgust_norm, fear_norm, joy_norm, sadness_norm, surprise_norm

+----------+-----+--------------------+
|prediction|label|            features|
+----------+-----+--------------------+
|       0.0|  0.0|(68,[0,1,2,3,4,5,...|
|       0.0|  0.0|(68,[0,1,2,3,4,5,...|
|       2.0|  2.0|(68,[0,1,2,3,4,5,...|
|       0.0|  0.0|(68,[0,1,2,3,4,5,...|
|       0.0|  0.0|(68,[0,1,2,3,4,5,...|
+----------+-----+--------------------+
only showing top 5 rows



# Model 2 (KNN Music Recommender)

In [81]:
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.sql import functions as F
from pyspark.sql.types import FloatType
from pyspark.sql import DataFrame

# Step 1: Aggregate features by Uri, retaining Title and Artist
model2_df = transformed_df.groupBy("Uri").agg(
    F.first("Title").alias("Title"),
    F.first("Artist").alias("Artist"),
    F.avg("danceability").alias("danceability"),
    F.avg("energy").alias("energy"),
    F.avg("loudness").alias("loudness"),
    F.avg("acoustics").alias("acoustics"),
    F.avg("instrumentalness").alias("instrumentalness"),
    F.avg("liveliness").alias("liveliness"),
    F.avg("valence").alias("valence"),
    F.avg("tempo").alias("tempo")
)

audio_features = ['danceability', 'energy', 'loudness', 'acoustics', 
                  'instrumentalness', 'liveliness', 'valence', 'tempo']
model2_df = model2_df.fillna(0, subset=audio_features)
# Combine audio features into a single vector column
assembler = VectorAssembler(inputCols=audio_features, outputCol="features")
data_with_features = assembler.transform(model2_df)  # Replace 'data' with your PySpark DataFrame



In [82]:
# Step 2: Standardize the features
scaler = StandardScaler(inputCol="features", outputCol="scaled_features")
scaler_model = scaler.fit(data_with_features)
data_scaled = scaler_model.transform(data_with_features)

24/11/14 15:46:47 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Country, Uri, danceability, energy, loudness, acoustics, instrumentalness, liveliness, valence, tempo
 Schema: Country0, Uri, danceability, energy, loudness, acoustics, instrumentalness, liveliness, valence, tempo
Expected: Country0 but found: Country
CSV file: file:///Users/nate/Desktop/FALL24%202/ENSF612/Project/Music_Analytics_and_Recommendation/Music_Database.csv


In [83]:
# Step 3: Define the recommendation function including Title and Artist in the output
def get_recommendations(df: DataFrame, target_uri: str, n_recommendations: int = 5) -> DataFrame:
    """
    Get song recommendations based on Euclidean distance to a target song by Uri.

    Parameters:
    df (DataFrame): The PySpark DataFrame containing scaled features.
    target_uri (str): The Uri of the target song.
    n_recommendations (int): Number of recommendations to retrieve.

    Returns:
    DataFrame: A PySpark DataFrame of recommended songs including Uri, Title, and Artist.
    """
    # Extract the feature vector of the target song by Uri
    target_song = df.where(F.col("Uri") == target_uri).select("scaled_features").first()
    
    # Check if target_song exists
    if not target_song:
        raise ValueError(f"Uri {target_uri} not found in the dataset.")
    
    target_vector = target_song["scaled_features"]

    # Define a UDF to calculate Euclidean distance
    def euclidean_distance(v1, v2):
        return float(v1.squared_distance(v2)) ** 0.5

    distance_udf = F.udf(lambda v: euclidean_distance(target_vector, v), FloatType())

    # Calculate distances and get the closest songs
    df_with_distances = df.withColumn("distance", distance_udf(F.col("scaled_features")))
    recommendations = (df_with_distances
                       .filter(F.col("Uri") != target_uri)  # Exclude the target song itself
                       .select("Uri", "Title", "Artist", "distance")  # Include Uri, Title, Artist
                       .orderBy("distance")
                       .limit(n_recommendations))
    
    return recommendations

In [84]:
# Example usage:
recommendations_df = get_recommendations(data_scaled, target_uri="https://open.spotify.com/track/6FyRXC8tJUh863JCkyWqtk", n_recommendations=5)
recommendations_df.show()

24/11/14 15:47:06 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Country, Uri, danceability, energy, loudness, acoustics, instrumentalness, liveliness, valence, tempo
 Schema: Country0, Uri, danceability, energy, loudness, acoustics, instrumentalness, liveliness, valence, tempo
Expected: Country0 but found: Country
CSV file: file:///Users/nate/Desktop/FALL24%202/ENSF612/Project/Music_Analytics_and_Recommendation/Music_Database.csv
24/11/14 15:47:08 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: Country, Uri, Title, Artist, danceability, energy, loudness, acoustics, instrumentalness, liveliness, valence, tempo
 Schema: Country0, Uri, Title, Artist, danceability, energy, loudness, acoustics, instrumentalness, liveliness, valence, tempo
Expected: Country0 but found: Country
CSV file: file:///Users/nate/Desktop/FALL24%202/ENSF612/Project/Music_Analytics_and_Recommendation/Music_Database.csv
/Users/nate/opt/anaconda3/envs/myenv/lib/pyt

+--------------------+------------------+--------------------+----------+
|                 Uri|             Title|              Artist|  distance|
+--------------------+------------------+--------------------+----------+
|https://open.spot...|        adan y eva|        Paulo Londra|0.04859169|
|https://open.spot...|la jeepeta - remix|Nio Garcia - Anue...|0.43462214|
|https://open.spot...|la jeepeta - remix|Nio Garcia - Anue...|0.44388047|
|https://open.spot...|       fuego lento|          Drake Bell| 0.5159253|
|https://open.spot...|     supuestamente|    Ozuna - Anuel AA|0.57871056|
+--------------------+------------------+--------------------+----------+

